 - If you know Numpy*,

In [4]:
import numpy as np
a = np.zeros((2,2)); b = np.ones((2,2))
np.sum(b, axis=1)

array([ 2.,  2.])

In [5]:
a.shape

(2, 2)

In [9]:
np.reshape(a,(1,4))

array([[ 0.,  0.,  0.,  0.]])

 - Repeat in TensorFlow

In [17]:
import tensorflow as tf
sess = tf.Session()

a = tf.zeros((2, 2)); b = tf.ones((2, 2))
sess.run(tf.reduce_sum(b,0))

array([ 2.,  2.], dtype=float32)

In [19]:
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [21]:
sess.run(tf.reshape(a, (1, 4)))

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

## sess.run()

In [31]:
sess = tf.Session()
a = np.zeros((2, 2)); ta = tf.zeros((2, 2))

print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [33]:
# TensorFlow first defines a computation graph that has no numerical value until evaluated.(very similar concept to Theano)
print(ta)

Tensor("zeros_8:0", shape=(2, 2), dtype=float32)


In [34]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


## Computation Graph
 - In TensorFlow program, it first assembles a graph, and uses a session to execute ops in the graph

In [36]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

sess = tf.Session()
print(sess.run(c))

30.0


## TensorFlow Session

 - A Session object encapsulates the environment in which operation and tensor objects are evaluated
 - Both type of the session creation has same behavior

In [40]:
sess = tf.Session()
print(sess.run(c))


with tf.Session() as sess:
    print(sess.run(c))
    print(c.eval())

30.0
30.0
30.0


## TensorFlow Variables
 - All tensors used previously were constant tensors
 - To train a model, we need variable type to hold and update values

In [46]:
w = tf.Variable(tf.zeros((2,2)),name ="weight")
with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[ 0.  0.]
 [ 0.  0.]]


## Variables need initialization!

In [45]:
w = tf.Variable(tf.random_normal([5, 2], stddev=0.1), name="weight")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[-0.01294442  0.01419749]
 [ 0.06661883 -0.00926303]
 [ 0.00124434  0.07218869]
 [ 0.13402466 -0.05318997]
 [ 0.04164162 -0.09472268]]


## Updating Variables

In [47]:
state = tf.Variable(0, name="counter")
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


## Fetching Variables

In [54]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3)
mul  = tf.multiply(x1, temp)

with tf.Session() as sess:
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


## TensorFlow Placeholder
 - All previous examples have manually defined tensors
  - How can we input externlal data into TensorFlow?
  - Most simple way is using tf.placeholder and feed_dict

In [61]:
a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b)

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a:2, b:3}))
    print(sess.run(mul, feed_dict={a:2, b:3}))

5
6


## TensorFlow Placeholder

In [73]:
# using tf.constant
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    result = sess.run(product)
    print(result)
    
# using placeholder
import numpy as np
matrix1 = tf.placeholder(tf.float32, [1, 2])
matrix2 = tf.placeholder(tf.float32, [2, 1])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.],[2.]])
    result = sess.run(product, feed_dict = {matrix1: mv1, matrix2: mv2})
    print(result)

[[ 12.]]
[[ 12.]]


## Example - MNIST with MLP

In [3]:
import tensorflow as tf

# Import MINST data

from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data. read_data_sets ( "/tmp/data/" , one_hot= True )

learning_rate = 0.001 
max_steps = 15000 
batch_size = 128

x = tf. placeholder (tf.float32, [ None , 784 ]) 
y = tf. placeholder (tf.float32, [ None , 10 ])

def MLP (inputs):
    W_1 = tf. Variable (tf. random_normal ([ 784 , 256 ])) 
    b_1 = tf. Variable (tf. zeros ([ 256 ]))

    W_2 = tf. Variable (tf. random_normal ([ 256 , 256 ]))
    b_2 = tf. Variable (tf. zeros ([ 256 ]))

    W_out = tf.Variable (tf. random_normal ([ 256 , 10 ]))
    b_out = tf. Variable (tf. zeros ([ 10 ]))

    h_1 = tf.add (tf.matmul (inputs, W_1), b_1)
    h_1 = tf.nn.relu (h_1)

    h_2 = tf.add (tf.matmul (h_1, W_2), b_2) 
    h_2 = tf.nn.relu (h_2)

    out = tf.add (tf.matmul (h_2, W_out), b_out)
    return out

net = MLP (x)


# define loss and optimizer 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y))
opt = tf.train.AdamOptimizer(learning_rate).minimize (loss_op)

# initializing the variables
init_op = tf.global_variables_initializer() 
sess = tf.Session () 
sess.run(init_op) 

# train model

for step in range (max_steps):

    batch_X, batch_y = mnist.train.next_batch(batch_size) 
    _, loss = sess.run([opt, loss_op], feed_dict={x: batch_X, y: batch_y})  # _ 이거는 빈 상자 같은 뜻인 듯 데이터 처리 후 버릴때 쓰는 것 ? 

    if (step+ 1) % 1000 == 0 :                                         #1000마다 한번씩 출력되도록
        print (" [{}/{}] loss:{:.3f} ". format (step+ 1 , max_steps, loss))
        
print (" Optimization Finished! ") 

# test modelb
correct_prediction = tf.equal (tf.argmax(net, 1), tf.argmax (y, 1)) 

# calculate accuracy

accuracy = tf.reduce_mean(tf.cast (correct_prediction, tf.float32)) 
print ("Train accuracy: {:.3f} ".format (sess. run (accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels}))) 
print ("Test accuracy: {:.3f} ".format (sess. run (accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
 [1000/15000] loss:35.415 
 [2000/15000] loss:12.680 
 [3000/15000] loss:14.525 
 [4000/15000] loss:3.504 
 [5000/15000] loss:2.866 
 [6000/15000] loss:3.122 
 [7000/15000] loss:2.622 
 [8000/15000] loss:0.818 
 [9000/15000] loss:1.386 
 [10000/15000] loss:0.000 
 [11000/15000] loss:0.000 
 [12000/15000] loss:0.373 
 [13000/15000] loss:0.000 
 [14000/15000] loss:0.000 
 [15000/15000] loss:0.000 
 Optimization Finished! 
Train accuracy: 0.998 
Test accuracy: 0.956 


## Training a model
 - tf.nn.softmax_cross_entropy_with_logits
 is for calculate cross entropy loss between predictions and labels (instead, you can make own loss function)
 
 - tf.train.####Optimizer create an optimizer such as SGD, RMSProp, Adam etc..*
 
 - tf.train.####Optimizer.minimize(loss_op) adds optimization operation to computation graph
 
 - we don't have to be concern about computing gradients and updating variables

## Variable Scope
 - Complicated TensorFlow models can have hendreds of variables
  - tf.variable_scope() provides simple name-spacing to avoid clashes
  - tf.get_variable() creates/accesses variables from within a variable scope

## tf.variable_scope()
 - Variable scope is a simple type of name-spacing that adds prefixes to variable names within scope

In [8]:
var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        var2 = tf.Variable([1], name="var")
        var3 = tf.Variable([1], name="var")
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var_4:0
var2: foo_4/bar/var:0
var3: foo_4/bar/var_1:0


## tf.get_variable()
 - tf.Variable doesn't provide variable reuse
  - Variable reuse is necessary to implement RNN of Recursive NN

In [16]:
import tensorflow as tf
tf.reset_default_graph() #주피터 노트북과 같은 환경에서 코드를 실행하게되면, 주피터 커널을 리스타트하지않는 이상 변수들의 컨택스트(상황정보?)
                         #컨택스트가 그대로 유지되기 때문에, 아래의 코드를 tf.reset_default_graph()없이 두번 이상 실행하게되면 에러가 난다.
                         #위치는 상관없는 것 같다 아래에 있어도 괜찮음. 

var1 = tf.get_variable("var1", [1])


with tf.variable_scope("foo1"):
    with tf.variable_scope("bar") as scp:
        var2 = tf.get_variable("var", [1]) 
        scp.reuse_variables() # allow reuse variables
        var3 = tf.get_variable("var", [1])
        
print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))


var1: var1:0
var2: foo1/bar/var:0
var3: foo1/bar/var:0


## tf.get_variable()
 - Behavior depends on whether variable reuse enabled
 - Case 1:reuse flag set to False
  - Create new variable and return
  - Raise ValueError if there exists variable with given name
 - Case 2:reuse flag set to True
  - Search for existinf variable with given name
  - Raise ValueError if not found

## Parameter sharing

In [70]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        scp.reuse_variables()
        var1 = tf.get_variable("var", [1])  
        var2 = tf.get_variable("var", [1])
        
    with tf.variable_scope("bar", reuse=True):
        var3 = tf.get_variable("var", [1])

print("var1: {}".format(var1.name)) 
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: foo/bar/var_2:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


## Wrappers

In [20]:
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W", [ksize, ksize, bottom_shape, numfilter], initializer=tf.constant_initializer(0))
        b = tf.get_variable("b", [num_filter], initializer = tf.constant_initializer(0))
        x = tf.nn.conv2d(bottom, W, strides = [1, stride, stride, 1], padding = padding)
        x = tf.nn.relu(tf.nn.bias_add(x,b))
        
    return x

def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope= None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1,ksize,ksize,1],stride=[1,stride,stride,1], padding=padding)
        return pool
    
    
def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims], initializer = he_init)
        b = tf.get_variable("b", [num_dims], initializer = tf.constant_initializer(0))
        
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope = "fc")
        relu = tf.nn.relu(out)
        
    return relu

keep_prob = tf.placeholder(tf.float32, None)

def conv_net(x, keep_prob):
    x = tf.reshape(x, shape = [-1,28,28,1])
    
    conv1 = conv(x, 32, 5, scope = "conv_1")
    conv1 = ma8
    
    xpool(conv1, scope ="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope = "conv_2")
    conv2 = maxpool(conv2, scope ="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope= "fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope= "out" )
    return out

## Save/Restore
 - My save wrapper function in my custom network class

In [24]:
def save(self, ckpt_dir, global_step=None):
    if self.config.get("saver") is None:
        self.config["saver"] = \
            tf.train.Saver(max_to_keep = 30)
    
    saver = self.config["saver"]
    sess  = self.config["sess"]
    
    dirname = os.path.join(ckpt_dir, self.name)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    saver.save(sess, dirname, global_step)
    

## Save/Restore
 - My restore wrapper function

In [25]:
def load_latest_checkpoint(self, ckpt_dir, exclude=None):
    path = tf.train.latest_checkpoint(ckpt_dir)
    if path is None:
        raise AssertionError("No ckpt exists in {0}.".format(ckpt_dir))
        
        print("Load {} save file".format(path))
        self._load(path, exclude)


def load_from_path(self, ckpt_path, exclude = None):
    self._load(ckpt_path, exclude)
    
def _load(self, ckpt_path, exclude):
    init_fn = slim.assign_from_checkpoint_fn(ckpt_path, slim.get_variables_to_restore(exclude=exclude), ignore_missing_vars=True)
    init_fn(self.config["sess"])

In [3]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains model definitions for versions of the Oxford VGG network.
These model definitions were introduced in the following technical report:
  Very Deep Convolutional Networks For Large-Scale Image Recognition
  Karen Simonyan and Andrew Zisserman
  arXiv technical report, 2015
  PDF: http://arxiv.org/pdf/1409.1556.pdf
  ILSVRC 2014 Slides: http://www.robots.ox.ac.uk/~karen/pdf/ILSVRC_2014.pdf
  CC-BY-4.0
More information can be obtained from the VGG website:
www.robots.ox.ac.uk/~vgg/research/very_deep/
Usage:
  with slim.arg_scope(vgg.vgg_arg_scope()):
    outputs, end_points = vgg.vgg_a(inputs)
  with slim.arg_scope(vgg.vgg_arg_scope()):
    outputs, end_points = vgg.vgg_16(inputs)
@@vgg_a
@@vgg_16
@@vgg_19
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

slim = tf.contrib.slim


def vgg_arg_scope(weight_decay=0.0005):
  """Defines the VGG arg scope.
  Args:
    weight_decay: The l2 regularization coefficient.
  Returns:
    An arg_scope.
  """
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_regularizer=slim.l2_regularizer(weight_decay),
                      biases_initializer=tf.zeros_initializer()):
    with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
      return arg_sc


def vgg_a(inputs,
          num_classes=1000,
          is_training=True,
          dropout_keep_prob=0.5,
          spatial_squeeze=True,
          scope='vgg_a'):
  """Oxford Net VGG 11-Layers version A Example.
  Note: All the fully_connected layers have been transformed to conv2d layers.
        To use in classification mode, resize input to 224x224.
  Args:
    inputs: a tensor of size [batch_size, height, width, channels].
    num_classes: number of predicted classes.
    is_training: whether or not the model is being trained.
    dropout_keep_prob: the probability that activations are kept in the dropout
      layers during training.
    spatial_squeeze: whether or not should squeeze the spatial dimensions of the
      outputs. Useful to remove unnecessary dimensions for classification.
    scope: Optional scope for the variables.
  Returns:
    the last op containing the log predictions and end_points dict.
  """
  with tf.variable_scope(scope, 'vgg_a', [inputs]) as sc:
    end_points_collection = sc.name + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.max_pool2d],
                        outputs_collections=end_points_collection):
      net = slim.repeat(inputs, 1, slim.conv2d, 64, [3, 3], scope='conv1')
      net = slim.max_pool2d(net, [2, 2], scope='pool1')
      net = slim.repeat(net, 1, slim.conv2d, 128, [3, 3], scope='conv2')
      net = slim.max_pool2d(net, [2, 2], scope='pool2')
      net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='conv3')
      net = slim.max_pool2d(net, [2, 2], scope='pool3')
      net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv4')
      net = slim.max_pool2d(net, [2, 2], scope='pool4')
      net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv5')
      net = slim.max_pool2d(net, [2, 2], scope='pool5')
      # Use conv2d instead of fully_connected layers.
      net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout6')
      net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout7')
      net = slim.conv2d(net, num_classes, [1, 1],
                        activation_fn=None,
                        normalizer_fn=None,
                        scope='fc8')
      # Convert end_points_collection into a end_point dict.
      end_points = slim.utils.convert_collection_to_dict(end_points_collection)
      if spatial_squeeze:
        net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
        end_points[sc.name + '/fc8'] = net
      return net, end_points
vgg_a.default_image_size = 224


def vgg_16(inputs,
           num_classes=1000,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_16'):
  """Oxford Net VGG 16-Layers version D Example.
  Note: All the fully_connected layers have been transformed to conv2d layers.
        To use in classification mode, resize input to 224x224.
  Args:
    inputs: a tensor of size [batch_size, height, width, channels].
    num_classes: number of predicted classes.
    is_training: whether or not the model is being trained.
    dropout_keep_prob: the probability that activations are kept in the dropout
      layers during training.
    spatial_squeeze: whether or not should squeeze the spatial dimensions of the
      outputs. Useful to remove unnecessary dimensions for classification.
    scope: Optional scope for the variables.
  Returns:
    the last op containing the log predictions and end_points dict.
  """
  with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
    end_points_collection = sc.name + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=end_points_collection):
      net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
      net = slim.max_pool2d(net, [2, 2], scope='pool1')
      net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
      net = slim.max_pool2d(net, [2, 2], scope='pool2')
      net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
      net = slim.max_pool2d(net, [2, 2], scope='pool3')
      net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
      net = slim.max_pool2d(net, [2, 2], scope='pool4')
      net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
      net = slim.max_pool2d(net, [2, 2], scope='pool5')
      # Use conv2d instead of fully_connected layers.
      net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout6')
      net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout7')
      net = slim.conv2d(net, num_classes, [1, 1],
                        activation_fn=None,
                        normalizer_fn=None,
                        scope='fc8')
      # Convert end_points_collection into a end_point dict.
      end_points = slim.utils.convert_collection_to_dict(end_points_collection)
      if spatial_squeeze:
        net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
        end_points[sc.name + '/fc8'] = net
      return net, end_points
vgg_16.default_image_size = 224


def vgg_19(inputs,
           num_classes=1000,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_19'):
  """Oxford Net VGG 19-Layers version E Example.
  Note: All the fully_connected layers have been transformed to conv2d layers.
        To use in classification mode, resize input to 224x224.
  Args:
    inputs: a tensor of size [batch_size, height, width, channels].
    num_classes: number of predicted classes.
    is_training: whether or not the model is being trained.
    dropout_keep_prob: the probability that activations are kept in the dropout
      layers during training.
    spatial_squeeze: whether or not should squeeze the spatial dimensions of the
      outputs. Useful to remove unnecessary dimensions for classification.
    scope: Optional scope for the variables.
  Returns:
    the last op containing the log predictions and end_points dict.
  """
  with tf.variable_scope(scope, 'vgg_19', [inputs]) as sc:
    end_points_collection = sc.name + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=end_points_collection):
      net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
      net = slim.max_pool2d(net, [2, 2], scope='pool1')
      net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
      net = slim.max_pool2d(net, [2, 2], scope='pool2')
      net = slim.repeat(net, 4, slim.conv2d, 256, [3, 3], scope='conv3')
      net = slim.max_pool2d(net, [2, 2], scope='pool3')
      net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv4')
      net = slim.max_pool2d(net, [2, 2], scope='pool4')
      net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv5')
      net = slim.max_pool2d(net, [2, 2], scope='pool5')
      # Use conv2d instead of fully_connected layers.
      net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout6')
      net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout7')
      net = slim.conv2d(net, num_classes, [1, 1],
                        activation_fn=None,
                        normalizer_fn=None,
                        scope='fc8')
      # Convert end_points_collection into a end_point dict.
      end_points = slim.utils.convert_collection_to_dict(end_points_collection)
      if spatial_squeeze:
        net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
        end_points[sc.name + '/fc8'] = net
      return net, end_points
vgg_19.default_image_size = 224

# Alias
vgg_d = vgg_16
vgg_e = vgg_19

## Tensor Board
 - Very useful visualization tool
  - Graph visualization, loss/accuracy plot, weight histogram plot

## TensorBoard
 - Simple usage
    - Make a tf.summary.FileWriter to write summaries
    - Create summaries using tf.summary.###
    - Run summaries with sess.run(...)
    - Add summaries to FileWriter with add_summary API
    - Run command    tensorboard --logdir=## -host=## -port=##
    - Default port is 6006
    

In [10]:
import tensorflow as tf
slim = tf.contrib.slim

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

max_steps = 10000
batch_size = 128
lr = 0.001
keep_prob = 0.5
weight_decay = 0.0004
logs_path = "/tmp/tensorflow_logs/example"

def my_arg_scope(is_training, weight_decay):
    with slim.arg_scope([slim.conv2d],
                       activation_fn=tf.nn.relu,
                       weight_regularizer = slim.l2_regularizer(weight_decay),
                       weight_initializer = slim.variance_scaling_initializer(),
                       biases_initializer = tf.zeros_initializer,
                       stride=1, padding="SAME"):
        with slim.arg_scope([slim.dropout],
                           is_training = is_training) as arg_sc:
            return arg_sc
        
def my_net(x, keep_prob, outputs_collections ="my_net"):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    with slim.arg_scope([slim.conv2d, slim.max_pool2d],
                       outputs_collections = outputs_collections):
        net = slim.conv2d(x, 64, [3,3], scope="conv1")
        net = slim.max_pool2d(net, [2,2], scope="pool1")
        
        net = slim.conv2d(net, 128, [3,3], scope="conv2")
        net = slim.max_pool2d(net, [2,2], scope="pool2")
        
        net = slim.conv2d(net, 256, [3,3], scope="conv3")
        # global average pooling
        net = tf.reduce_mean(net, [1, 2], name="pool3",keep_dims=True)
        net = slim.dropout(net, keep_prob, scope = "dropout3")
        net = slim.conv2d(net, 1024, [1,1], scope="fc4")
        
        net = slim.dropout(net, keep_prob, scope = "dropout4")
        net = slim.conv2d(net, 10, [1,1],
                          activation_fn=None, scope="fc5")
        
    end_points= \
        slim.utils.convert_collection_to_dict(outputs_collections)
    return tf.reshape(net, [-1,10]), end_points

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with slim.arg_scope(my_arg_scope(is_training, weight_decay)):
    net, end_pts = my_net(x, keep_prob)
    pred = slim.softmax(net, scope="prediction")
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = cls_loss + reg_loss
    
with tf.variable_scope("Admam"):
    opt= tf.train.AdamOptimizer(lr)
    # Op to calculate every variable gradient
    grads = tf.gradients(loss_op, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    # Op to update all variables according to their gradient
    apply_grads = opt.apply_gradients(grads_and_vars=grads)
    
with tf.variable_scope("accuracy"):
    correct_op= tf.equal(tf.argmax(net,1), tf.argmax(y,1))
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))
    
    
# Create a summary to monitor loss and accuracy
summ_loss = tf.summary.scalar("loss", loss_op)
summ_acc = tf.summary.scalar("accuracy_test", acc_op)

# Create summary to visualize weights and grads
for var in tf.trainable_varialbes():
    tf.summary.histogram(var.name, var, collections=["my_summ"])
for grad, var in grads:
    tf.summary.histogram(var.name + "/gradient", grad, collections=["my_summ"])
for grad, var in grads:
    tf.summary.histogram(var.name + "/gradient", grad, collections=["my_summ"])

summ_wg = tf.summary.merge_all(key="my_summ")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

for step in range(max_steps):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    _, loss, plot_loss, plot_wg = sess.run([apply_grads, loss_op, summ_loss, summ_wg],
                                          feed_dict = {x: batch_X, y: batch_y, is_training: True})
    summary_writer.add_summary(plot_loss, step)

print("Optimization Finished!")

test_acc = sess.run(acc_op, feed_dict={x: mnist.test.images, 
                                      y:mnist.test.labels,
                                      is_training: False})
print("Test accuracy: {:.3f}".format(test_acc))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


TypeError: convolution() got an unexpected keyword argument 'weight_regularizer'